In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from ipynb.fs.full.utility_functions import gen_data_dict, get_data, get_cumsum, error_count, write_to_csv, opart, SquaredHingeLoss

np.random.seed(4)
torch.manual_seed(4)

In [2]:
def normalize_data(data):
    mean = data.mean(dim=0)
    std  = data.std(dim=0)
    norm_data = (data - mean) / std
    return norm_data

In [3]:
# data
chosen_feature = ['std_deviation', 'count', 'sum_diff']
data = torch.FloatTensor(pd.read_csv('learning_data/seq_features.csv').iloc[:, 1:][chosen_feature].to_numpy())
data = normalize_data(data)

target_df_1 = pd.read_csv('learning_data/target_lambda_fold1.csv')
target_df_2 = pd.read_csv('learning_data/target_lambda_fold2.csv')

targets_low_1  = target_df_1.iloc[:, 1:2].to_numpy()
targets_high_1 = target_df_1.iloc[:, 2:3].to_numpy()
targets_low_2  = target_df_2.iloc[:, 1:2].to_numpy()
targets_high_2 = target_df_2.iloc[:, 2:3].to_numpy()

targets_low_1  = torch.FloatTensor(targets_low_1)
targets_high_1 = torch.FloatTensor(targets_high_1)
targets_low_2  = torch.FloatTensor(targets_low_2)
targets_high_2 = torch.FloatTensor(targets_high_2)

In [4]:
# Define the deep learning model
class MyModel(nn.Module):
    def __init__(self, input_size):
        super(MyModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 8)
        self.fc2 = nn.Linear(8, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [5]:
# Instantiate the model, define custom loss function, and optimizer
model1 = MyModel(input_size = data.shape[1])
model2 = MyModel(input_size = data.shape[1])

squared_hinge_loss = SquaredHingeLoss()
optimizer1 = optim.Adam(model1.parameters(), lr=0.001)
optimizer2 = optim.Adam(model2.parameters(), lr=0.001)

# Training loop
record_loss1_test = []
record_loss2_test = []
min_loss_1_test = float('inf')
min_loss_2_test = float('inf')
for epoch in range(1000):
    # Forward pass
    outputs1 = model1(data)
    outputs2 = model2(data)
    
    # Compute the custom loss
    loss_1 = squared_hinge_loss(outputs1, targets_low_1, targets_high_1)
    loss_2 = squared_hinge_loss(outputs2, targets_low_2, targets_high_2)
    loss_1_test = squared_hinge_loss(outputs1, targets_low_2, targets_high_2)
    loss_2_test = squared_hinge_loss(outputs2, targets_low_1, targets_high_1)
    
    # Backward pass and optimization
    optimizer1.zero_grad()
    loss_1.backward()
    optimizer1.step()

    optimizer2.zero_grad()
    loss_2.backward()
    optimizer2.step()

    # record
    record_loss1_test.append(loss_1_test.item())
    record_loss2_test.append(loss_2_test.item())

    # save models
    if loss_1_test < min_loss_1_test:
        min_loss_1_test = loss_1_test
        torch.save(model1.state_dict(), 'saved_models/model1_best.pth')
    
    if loss_2_test < min_loss_2_test:
        min_loss_2_test = loss_2_test
        torch.save(model2.state_dict(), 'saved_models/model2_best.pth')
    
    # Print the loss every 100 epochs
    if (epoch) % 100 == 0:
        print(f'Epoch {epoch:5d}, Loss_1: {loss_1.item():8.4f}, Loss_1_test: {loss_1_test.item():8.4f}, Loss_2: {loss_2.item():8.4f}, Loss_2_test: {loss_2_test.item():8.4f}')

Epoch     0, Loss_1:   1.2396, Loss_1_test:   1.6387, Loss_2:   1.8116, Loss_2_test:   1.4346
Epoch   100, Loss_1:   1.0846, Loss_1_test:   1.4331, Loss_2:   1.5040, Loss_2_test:   1.2265
Epoch   200, Loss_1:   1.0410, Loss_1_test:   1.3202, Loss_2:   1.3852, Loss_2_test:   1.2062
Epoch   300, Loss_1:   1.0247, Loss_1_test:   1.2953, Loss_2:   1.3280, Loss_2_test:   1.1782
Epoch   400, Loss_1:   1.0151, Loss_1_test:   1.3015, Loss_2:   1.2668, Loss_2_test:   1.1328
Epoch   500, Loss_1:   1.0125, Loss_1_test:   1.3071, Loss_2:   1.2239, Loss_2_test:   1.1104
Epoch   600, Loss_1:   1.0113, Loss_1_test:   1.3095, Loss_2:   1.2077, Loss_2_test:   1.1031
Epoch   700, Loss_1:   1.0102, Loss_1_test:   1.3128, Loss_2:   1.2019, Loss_2_test:   1.0994
Epoch   800, Loss_1:   1.0094, Loss_1_test:   1.3135, Loss_2:   1.1979, Loss_2_test:   1.0974
Epoch   900, Loss_1:   1.0087, Loss_1_test:   1.3133, Loss_2:   1.1945, Loss_2_test:   1.0982


In [6]:
# Load model1
model1 = MyModel(input_size=data.shape[1])
model1.load_state_dict(torch.load('saved_models/model1_best.pth'))
model1.eval()  # Set the model to evaluation mode

# Load model2
model2 = MyModel(input_size=data.shape[1])
model2.load_state_dict(torch.load('saved_models/model2_best.pth'))
model2.eval()  # Set the model to evaluation mode

MyModel(
  (fc1): Linear(in_features=3, out_features=8, bias=True)
  (fc2): Linear(in_features=8, out_features=1, bias=True)
)

In [7]:
print("loss for fold 2: ",squared_hinge_loss(model1(data), targets_low_2, targets_high_2).item())
print("loss for fold 1: ",squared_hinge_loss(model2(data), targets_low_1, targets_high_1).item())

loss for fold 2:  1.2906907796859741
loss for fold 1:  1.0973103046417236


In [8]:
with torch.no_grad():
    ldas1 = model1(data).numpy().reshape(-1)
    ldas2 = model2(data).numpy().reshape(-1)

In [9]:
seqs   = gen_data_dict('sequence_label_data/signals.gz')
labels = gen_data_dict('sequence_label_data/labels.gz')

header = ['sequenceID', 'fold_1_total_labels', 'fold_2_total_labels', 'fold_1_errs', 'fold_2_errs']

for i in range(len(seqs)):
    # generate data
    sequence, neg_start_1, neg_end_1, pos_start_1, pos_end_1, neg_start_2, neg_end_2, pos_start_2, pos_end_2 = get_data(i, seqs=seqs, labels=labels)
    sequence_length = len(sequence)-1

    # vectors of cumulative sums
    y, z = get_cumsum(sequence)

    # get total labels
    fold1_total_labels = len(neg_start_1) + len(pos_start_1)
    fold2_total_labels = len(neg_start_2) + len(pos_start_2)

    # run each lambda and record it into csv file
    row  = [i, fold1_total_labels, fold2_total_labels]

    chpnt_fold1 = opart(10**ldas2[i], sequence)
    chpnt_fold2 = opart(10**ldas1[i], sequence)

    err_1 = error_count(chpnt_fold1, neg_start_1, neg_end_1, pos_start_1, pos_end_1)
    err_2 = error_count(chpnt_fold2, neg_start_2, neg_end_2, pos_start_2, pos_end_2)
    
    row.append(sum(err_1))
    row.append(sum(err_2))

    write_to_csv('learning_output/deep.csv', header, row)